<!--_____
***

<img width='700' src="https://user-images.githubusercontent.com/8030363/108961534-b9a66980-7634-11eb-96e2-cc46589dcb8c.png" style="vertical-align:middle">Application of PheKnowLator to the construction of RNA-centered Knowledge Graph from the integration of -->

# <p style="text-align: center;">Construction of a simplified RNA-centered Knowledge Graph</p>

***
***

**Authors:** [E. Cavalleri](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=emanuele.cavalleri@unimi.it), [T.J. Callahan](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=callahantiff@gmail.com)

**GitHub Repositories:** [testRNA-KG](https://github.com/emanuelecavalleri/testRNA-KG), [PheKnowLator](https://github.com/callahantiff/PheKnowLator/)  
<!--- **Release:** **[v2.0.0](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)** --->
  
<br>  

**Objective:** Knowledge graphs provide meaningful ways to integrate heterogeneous biological data and represent complex biological mechanisms. 
[PheKnowLator](https://github.com/callahantiff/PheKnowLator) is a system that support the user in the acquisition of biomedical entities from different kinds of data sources and their representation in terms of a biomedical knowledge graph that models unbiased molecular mechanisms as prescribed by domain ontologies. In this notebook we wish to show the use of PheKnowLator for the generation of a RNA-based knowledge graph that involves **gene-disease**, **gene-miRNA**, and **miRNA-disease** relationships. These relationships are made available from the following public repositories (we reduced row size):
- [gene-disease](https://storage.googleapis.com/pheknowlator/current_build/data/original_data/curated_gene_disease_associations.tsv), provided by PheKnowLator ecosystem itself
- [gene-miRNA](https://dianalab.e-ce.uth.gr/downloads/tarbase_v8_data.tar.gz), provided by TarBase
- [miRNA-disease](http://watson.compbio.iupui.edu:8080/miR2Disease/download/AllEntries.txt), provided by miR2Disease

The construction of the knowledge graph should be compliant with the following Ontologies:
- Mondo Disease Ontology ([MONDO](https://obofoundry.org/ontology/mondo.html))
- Relations Ontology ([RO](https://obofoundry.org/ontology/ro.html))
- Non-Coding RNA Ontology ([NCRO](https://obofoundry.org/ontology/ncro.html))

All the data required for the execution of this notebook can be downloaded from this [testRNA-KG/resources](https://drive.google.com/drive/folders/1ClpodnqNc2512z-2ImKCHr28M7w3IJRR) Google Drive repository.

<a target="_blank" href="https://user-images.githubusercontent.com/33032169/225636670-056a7774-f3d6-4aee-84b1-4f462c3cf33a.png"> <img src="https://user-images.githubusercontent.com/33032169/225636670-056a7774-f3d6-4aee-84b1-4f462c3cf33a.png"></a> 

(*Click Figure to Enlarge Image in Current Browser Tab*)

<br>

***
***

## Notebook Purpose
**Wiki Page:** **[`Current PheKnowLator release`](https://github.com/callahantiff/PheKnowLator/wiki/)**

<br>

**Purpose:** This notebook serves as a `main` file for the RNA-KG construction based on PheKnowLator project. This scripts walks through this program step-by-step and generates the knowledge graph shown above. Please see the [PheKnowLator README](https://github.com/callahantiff/PheKnowLator/blob/master/README.md) for more information.

<br>

**Assumptions:**
1. All downloaded and generated data sources are provided through [this](https://drive.google.com/drive/folders/14oXaqKbH2JkQ_nEqCRWB3T26ESaWoySK) dedicated Google Drive repository.  

2. Make sure that the following input documents have been constructed (see the [PheKnowLator Dependencies Wiki](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies) for more information):  
  - [`resource_info.txt`](https://github.com/emanuelecavalleri/testRNA-KG/blob/master/resources/resource_info.txt)
  - [`ontology_source_list.txt`](https://github.com/emanuelecavalleri/testRNA-KG/blob/master/resources/ontology_source_list.txt)
  - [`edge_source_list.txt`](https://github.com/emanuelecavalleri/testRNA-KG/blob/main/resources/edge_source_list.txt)   

3. Download [`RELATIONS_LABELS.txt`](https://github.com/emanuelecavalleri/testRNA-KG/blob/main/resources/relations_data/RELATIONS_LABELS.txt) relations label file prior to running the scripts. Please see [PheKnowLator Dependencies Wiki](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data) for more information on how it is generated.

4. Select a knowledge graph construction method (i.e. `instance-based` or `subclass-based`).  

<br>

***
### Table of Contents
***
The three primary steps involved in building a knowledge graph are `Downloading Data Sources`, `Creating Edge Lists`, and `Building the knowledge graphs`.

* [Data Sources](#data-source)  
* [Create Edge Lists](#create-edges)  
* [Build Knowledge Graph](#build-kg)  

***

***

_____
### Set-Up Environment

In [1]:
# import needed libraries
import glob
import json
import pandas
import ray
import time

# import module
from pkt_kg.downloads import OntData, LinkedData
from pkt_kg.edge_list import CreatesEdgeList
from pkt_kg.knowledge_graph import FullBuild, PartialBuild, PostClosureBuild

2023-10-18 15:27:55,266	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


***
## Download Data Sources <a class="anchor" id="data-source"></a>

**Wiki Page:** **[`Dependencies`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies)**  

**Purpose:**
This portion of this portion of the algorithm is to download:
1. [Download Ontology Data](#download-ontology-data)  
2. [Download Edge Data](#download-edge-data)   

<br>

**Input Files:**
  - [`resource_info.txt`](https://github.com/emanuelecavalleri/testRNA-KG/blob/master/resources/resource_info.txt)
  - [`ontology_source_list.txt`](https://github.com/emanuelecavalleri/testRNA-KG/blob/master/resources/ontology_source_list.txt)
  - [`edge_source_list.txt`](https://github.com/emanuelecavalleri/testRNA-KG/blob/master/resources/edge_source_list.txt)

<br>

**Assumption:** All sources used to construct our knowledge graph need to be preprocessed and ready to download prior to running this code. All mapping, filtering, and label data have been generated prior to this step. For assistance with creating these datasets, see the [`RNA-KG_Preparation.ipynb`](https://github.com/emanuelecavalleri/testRNA-KG/blob/main/notebooks/RNA-KG_Preparation.ipynb) Jupyter Notebook.

***
***
### Ontology Data  <a class="anchor" id="download-ontology-data"></a>
Ontologies are the core data structure used when building PheKnowLator.

In [2]:
ont = OntData('resources/ontology_source_list.txt', 'resources/resource_info.txt')

ont.parses_resource_file()

In [3]:
ont.data_files = ont.source_list
ont.generates_source_metadata()


*** Generating Metadata ***



100%|██████████| 2/2 [00:00<00:00, 13819.78it/s]


In [4]:
ont._writes_source_metadata_locally()

100%|██████████| 2/2 [00:00<00:00, 14098.50it/s]


In [5]:
ont.resource_info

['gene-disease|;;|entity-class|RO_0003302|http://www.ncbi.nlm.nih.gov/gene/|http://purl.obolibrary.org/obo/|t|0;4|1:./resources/processed_data/DISEASE_MONDO_MAP.txt|10;>=;1.0|None',
 'gene-miRNA|;;|entity-entity|RO_0002434|http://www.ncbi.nlm.nih.gov/gene/|https://www.mirbase.org/hairpin/|t|0;2|0:./resources/processed_data/ENSEMBL_GENE_ENTREZ_GENE_MAP.txt;1:./resources/processed_data/MIRBASE_ID_ACCESSION_MAP.txt|None|None',
 'miRNA-disease|;;|entity-class|RO_0003302|https://www.mirbase.org/hairpin/|http://purl.obolibrary.org/obo/|t|1;0|0:./resources/processed_data/MIRBASE_ID_ACCESSION_MAP.txt;1:./resources/processed_data/DISEASE_DOID_MONDO_Map.txt|None|None']

<br>

### Edge Data   <a class="anchor" id="download-edge-data"></a>
In PheKnowLator, classes are nodes that originate from ontologies. Class data sources are Linked Data sources that are used to create edges in the knowledge graph and thus can connect to other class data sources. Sometimes we want to add data that is not already part of an ontology. In that case, data either be added as an `instance` of an existing ontology class or as its own `owl:class` by being added to the knowledge graph as a `subclass` of an existing `owl:class`.

In [6]:
edges = LinkedData('resources/edge_source_list.txt', 'resources/resource_info.txt')

edges.parses_resource_file()

In [7]:
edges.data_files = edges.source_list
edges.generates_source_metadata()


*** Generating Metadata ***



100%|██████████| 3/3 [00:00<00:00, 22590.51it/s]


In [8]:
edges._writes_source_metadata_locally()

100%|██████████| 3/3 [00:00<00:00, 21998.10it/s]


In [9]:
edges.source_list.keys()

dict_keys(['gene-disease', 'miRNA-disease', 'gene-miRNA'])

***

## Create Edge Lists <a class="anchor" id="create-edges"></a>

**Wiki Page:** **[`Data Sources`](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources)**

<br>

**Purpose:** The code below will take the dictionaries of processed data described above and use them to create edge lists for each of the edge types specified in the [`resource_info.txt`](https://github.com/emanuelecavalleri/testRNA-KG/blob/master/resources/resource_info.txt). Each edge list will be appended to a nested dictionary (see details below).

<br>

**Assumptions:**  
1. All `ontology` and `edge` data sources have been downloaded.   

2. All code in the [`RNA-KG_Preparation.ipynb`](https://github.com/emanuelecavalleri/testRNA-KG/blob/main/notebooks/RNA-KG_Preparation.ipynb) Jupyter Notebook has been run. This Notebook contains code needed to generate all mapping, filtering, and label data.

<br>

**Output:** `Master_Edge_List_Dict.json`. Below is an example of what the `Master Edge Dictionary` contains for each processed resource:  
```python
master_edges = {'miRNA-disease'  :
                {'source_labels'    : ';;',
                 'data_type'        : 'class-class',
                 'edge_relation'    : 'RO_0003302',
                 'uri'              : ('http://purl.obolibrary.org/obo/',
                                       'http://purl.obolibrary.org/obo/'),
                 'delimiter'        : 't',
                 'column_idx'       : '2;1',
                 'identifier_maps'  : '0:./[...]/hsa_NCRO_Map.txt;1:./[...]/DOID_MONDO_MAP.txt',
                 'evidence_criteria': 'None',
                 'filter_criteria'  : 'None',
                 'edge_list'        : ['...']}
```

***

In [10]:
import psutil
# set-up environment for parallel processing -- even if running program serially these steps are needed
cpus = 3#psutil.cpu_count(logical=True)
ray.init()

2023-10-11 18:14:18,256	INFO worker.py:1642 -- Started a local Ray instance.


Python version:,3.9.7
Ray version:,2.7.1


In [11]:
# combine data sources
combined_edges = dict(edges.data_files, **ont.data_files)
resource_info_loc = './resources/resource_info.txt'

# initialize edge dictionary class
master_edges = CreatesEdgeList(data_files=combined_edges, source_file=resource_info_loc)
master_edges.runs_creates_knowledge_graph_edges(source_file=resource_info_loc, data_files=combined_edges, cpus=cpus)

(CreatesEdgeList pid=772079) Finished Edge: miRNA-disease (miRNA = 219, disease = 80); 1148 unique edges
(CreatesEdgeList pid=772080) Finished Edge: gene-miRNA (gene = 2905, miRNA = 456); 3839 unique edges


**Preview Master Edge Data**  
Generate a table that includes each `edge-type`, its primary `relation`, example identifiers, and count of unique edges.

In [12]:
master_edges = json.load(open('resources/Master_Edge_List_Dict.json', 'r'))
master_edges.keys()

dict_keys(['gene-miRNA', 'miRNA-disease', 'gene-disease'])

In [13]:
# read in relation data
relation_data = open('./resources/relations_data/RELATIONS_LABELS.txt').readlines()
relation_dict = {x.split('\t')[0]: x.split('\t')[1].strip('\n') for x in relation_data}

# function to return key for any value
def get_key(my_dict, val):
    for key, value in my_dict.items():
        if val == value:
            return key
 
    return "key doesn't exist"

# print basic stats on each resource
edge_data = [[key,
              get_key(relation_dict, 'http://purl.obolibrary.org/obo/'+master_edges[key]['edge_relation']),
              ', '.join(master_edges[key]['edge_list'][0]),
              len(master_edges[key]['edge_list'])]
             for key in master_edges.keys()]

# convert dict to pandas df for nice printing
df = pandas.DataFrame(edge_data, columns = ['Edge Type', 'Relation', 'Example Edge', 'Unique Edges']) 
df                

(CreatesEdgeList pid=772078) Finished Edge: gene-disease (gene = 669, disease = 523); 761 unique edges


,Edge Type,Relation,Example Edge,Unique Edges
0,gene-miRNA,interacts with,"23350, MI0000734",3839
1,miRNA-disease,causes or contributes to condition,"MI0000066, MONDO_0004967",1148
2,gene-disease,causes or contributes to condition,"112609, MONDO_0008300",761


<br><br>

***

## Build Knowledge Graph  <a class="anchor" id="build-kg"></a>
**Wiki Pages:**  
- **[`KG-Construction`](https://github.com/callahantiff/PheKnowLator/wiki/KG-Construction)**  
- **[`relations-data`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data)**  
- **[`node-metadata`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata)** 

**Jupyter Notebooks:**  
- [`RNA-KG_Preparation.ipynb`](https://github.com/emanuelecavalleri/testRNA-KG/blob/master/notebooks/RNA-KG_Preparation.ipynb)  
[`Ontology_Cleaning.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Ontology_Cleaning.ipynb)  


<br>

**Assumptions:**  
- <u>Construction Approach</u>. If using the `subclass-based` construction approach, please make sure that a `pickled` dictionary mapping each non-ontology data node to an existing ontology class is created and added to the `./resources/knowledge_graph` directory (please see [here](https://github.com/callahantiff/PheKnowLator/tree/master/resources/knowledge_graphs#construction-method) for additional information).   
- <u>Relations Data</u>. If inverse relation data is going to be used to build the knowledge graph, that it has been generated and added to the `./resources/relations_data` directory (please see [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/relations_data/README.md) for additional information).  
- <u>Node Metadata</u>. If node metadata is going to be used to build the knowledge graph, that it has been generated and added to the `./resources/node_metadata` directory (please see [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/node_data/README.md) for additional information).  
- <u>Decoding OWL Semantics</u>. If decoding OWL-Semantics, please make sure to provide a list of owl:Property types to keep is created and added to the `./resources/knowledge_graph` directory (please see [here](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0) for additional information). 

<br>

**Input:** 
- `Master_Edge_List_Dict.json`  
- Directory of relations data sources - see [here](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data) for more information
- Directory of node data sources - see [here](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) for more information

<br>

**Output:** Please see [`Release v2.0.0 Wiki`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0) for access to all generated output files.   
- `Knowledge Graph` (`.owl` and Networkx MultiDiGraph `.pkl`)  
- `Class Instance URI-UUID Map` (if "instance" construction approach)   
- `Triple List - Integer`  
- `Triple List - Identifier`  
- `Node Integer-Identifier Map`  
- `Node Attribute Data`  

<br>

The process to build the knowledge graph is somewhat time consuming and can be broken into the following steps:  

1. Merge Ontologies. See [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/ontologies/README.md) for additional information on how to preprocess the ontologies prior to merging them.    

2. Create Edges. Add edge lists to merged ontologies.  

3. Add Inverse Relations and Node Data. See the [Dependencies](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies) Wiki page for details on how to construct these resources.  

4. Filter OWL Semantics. Filter the knowledge graph with the goal of removing all edges that contain entities that are needed to support owl semantics, but are not biologically meaningful (please see [here](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0) for additional information).

5. Save Edge Lists and Node Metadata. Several versions of the knowledge graph are saved, including: the full knowledge graph (`owl` or Networkx MultiDiGraph `pickle`), triple lists (i.e. integer index and identifier labeled edge lists with a dictionary that maps between the integer indices and node identifiers), and a file of metadata (i.e. identifiers, labels, synonyms, and descriptions) for all nodes in the knowledge graph.  

<br>

**‼ IMPORTANT:**  
- The file containing the merged ontologies is quite large and can take up to 10 minutes to read in.  This is not a limitation of the code directly, but rather a function of the [`RDFLib Library`](https://github.com/RDFLib). While there are other ways to read in this data, we maintain reliance on this library as it is the most user-friendly for non-RDF users.   
- If you'd like to include [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) when building the knowledge graph, please hold off on building the knowledge graph until you have generated the node data. For details on how to do this see the [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) section of the `Dependencies` Wiki Page or help for help with generating the data, please see the []() section of the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb) Jupyter Notebook.

***


In [14]:
# specify input arguments
build = 'full'
construction_approach = 'instance'
add_node_data_to_kg = 'yes'
add_inverse_relations_to_kg = 'yes'
decode_owl_semantics = 'yes'
kg_directory_location = './resources/knowledge_graphs'

In [ ]:
# construct knowledge graphs
if build == 'partial':
    kg = PartialBuild(construction=construction_approach,
                      node_data=add_node_data_to_kg,
                      inverse_relations=add_inverse_relations_to_kg,
                      decode_owl=decode_owl_semantics,
                      cpus=cpus,
                      write_location=kg_directory_location)
elif build == 'post-closure':
    kg = PostClosureBuild(construction=construction_approach,
                          node_data=add_node_data_to_kg,
                          inverse_relations=add_inverse_relations_to_kg,
                          decode_owl=decode_owl_semantics,
                          cpus=cpus,
                          write_location=kg_directory_location)
else:
    kg = FullBuild(construction=construction_approach,
                   node_data=add_node_data_to_kg,
                   inverse_relations=add_inverse_relations_to_kg,
                   decode_owl=decode_owl_semantics,
                   cpus=cpus,
                   write_location=kg_directory_location)

kg.construct_knowledge_graph()
ray.shutdown()

***

# Visualize the just obtained RNA-Knowledge Graph  <a class="anchor" id="build-kg"></a>
Here we show an example of the just generated KG, trying to exploit RDF functionalities. To begin, we need to convert the OWL KG into a Pandas dataframe.

In [ ]:
import pandas as pd
from rdflib import Graph, URIRef, Literal

# Load the N-Triples data into an RDFLib graph
g = Graph()
with open('./resources/knowledge_graphs/PheKnowLator_v3.1.1_full_instance_inverseRelations_OWL.nt', 'r') as f:
    data = f.read()
    g.parse(data=data, format='nt')

triples = []

# Iterate through the RDF graph and extract the triples
for subj, pred, obj in g:
    triples.append((subj, pred, obj))

# Convert the list of triples into a Pandas DataFrame
df = pd.DataFrame(triples, columns=['Subject', 'Predicate', 'Object'])
df

The OWL version of our Knowledge Graph appears disorganized when viewed through an ML lens. Let's explore if we can extract valuable information, focusing on the BRAF gene (http://www.ncbi.nlm.nih.gov/gene/673), as an example. Biologically speaking, BRAF is an oncogene (a gene can become an oncogene due to mutations or increased expression) the protein encoded by this gene is a Serine/Threonine [kinase](https://en.wikipedia.org/wiki/Kinase). The [B-Raf protein](https://www.uniprot.org/uniprotkb/P15056/entry) is involved in sending signals inside cells which are involved in directing cell growth. In 2002, it was shown to be mutated in some human cancers (https://doi.org/10.1038/nature00766).

In [ ]:
test_673 = df[df['Subject'].str.endswith("gene/673")]
test_673

BRAF *is_a* (http://www.w3.org/2000/01/rdf-schema#subClassOf) gene (http://purl.obolibrary.org/obo/SO_0000704), specifically a protein coding gene (http://purl.obolibrary.org/obo/SO_0001217).

In [ ]:
test_SO_0000704 = df[df['Subject'].str.endswith("SO_0000704")]
test_SO_0000704

A gene *is_a* biological region (http://purl.obolibrary.org/obo/SO_0001411).

Moreover, BRAF is object of the subsequent subjects.

In [ ]:
test_673o = df[df['Object'].str.endswith("gene/673")]
test_673o

*N7e6220e64f8b425ebb32b81544503792* MD5 hash is subject in specific relationships as follows.

In [ ]:
test_N7e = df[df['Subject'].str.contains("N7e6220e64f8b425ebb32b81544503792")]
test_N7e

N7e6220e64f8b425ebb32b81544503792 *corresponds to* (http://www.w3.org/1999/02/22-rdf-syntax-ns#type) BRAF gene (http://www.ncbi.nlm.nih.gov/gene/673) and *causes or contributes to condition* (http://purl.obolibrary.org/obo/RO_0003302), so we are going to focus on the middle row of this dataframe.

In [ ]:
test_N4f = df[df['Subject'].str.contains("N4fc8ac5cb2274b90809df4ad5de0c79b")]
test_N4f

[MONDO_0007893](http://purl.obolibrary.org/obo/MONDO_0007893) represents the *Noonan syndrome with multiple lentigines*, a rare multisystem genetic disorder characterized by lentigines, hypertrophic cardiomyopathy, short stature, pectus deformity, and dysmorphic facial features.

Furthermore, let us see if we can extrapolate information about interacting miRNA from our subgraph.

In [ ]:
df[df['Object'].str.endswith("gene/673")]

In [ ]:
test_Neaf = df[df['Subject'].str.contains("Neaf866f59ae041dd9be187c010995de0")]
test_Neaf

We are interested in the second relationship: *interacts with* ([RO_0002434](http://purl.obolibrary.org/obo/RO_0002434)).

In [ ]:
test_Nae9 = df[df['Subject'].str.contains("Nae9fca4cfea8405b8de4c023cd693d44")]
test_Nae9

BRAF interacts with the [MI0000454](https://www.mirbase.org/hairpin/MI0000454) miRNA precursor (aka *hsa-mir-137*).

Finally, we can save this RNA-KG subgraph to visualize it.

In [ ]:
test_673['Subject'] = '<' + test_673['Subject'].astype(str) + '>'
test_673['Predicate'] = '<' + test_673['Predicate'].astype(str) + '>'
test_673['Object'] = ['<http://purl.obolibrary.org/obo/SO_0001217>',
                      '<http://purl.obolibrary.org/obo/SO_0000704>',
                      '"BRAF"',
                      '<http://www.w3.org/2002/07/owl#Class>']

test_673

In [ ]:
test_SO_0000704['Subject'] = '<' + test_SO_0000704['Subject'].astype(str) + '>'
test_SO_0000704['Predicate'] = '<' + test_SO_0000704['Predicate'].astype(str) + '>'
test_SO_0000704['Object'] = ['<http://www.w3.org/2002/07/owl#Class>',
                             '"gene"',
                             '<http://purl.obolibrary.org/obo/SO_0001411>',
                             '_:N23642b58dd30483b813565e2741d4b77']

test_SO_0000704

In [ ]:
test_673o['Subject'] = '_:' + test_673o['Subject'].astype(str)
test_673o['Predicate'] = '<' + test_673o['Predicate'].astype(str) + '>'
test_673o['Object'] = '<' + test_673o['Object'].astype(str) + '>'

test_673o

In [ ]:
test_N7e['Subject'] = '_:' + test_N7e['Subject'].astype(str)
test_N7e['Predicate'] = '<' + test_N7e['Predicate'].astype(str) + '>'
test_N7e['Object'] = ['<http://www.w3.org/2002/07/owl#NamedIndividual>',
                     ]

test_N7e

In [ ]:
test_N4f['Subject'] = '_:' + test_N4f['Subject'].astype(str)
test_N4f['Predicate'] = '<' + test_N4f['Predicate'].astype(str) + '>'
test_N4f['Object'] = '<' + test_N4f['Object'].astype(str) + '>'

test_N4f

In [ ]:
test_Nae9['Subject'] = '_:' + test_Nae9['Subject'].astype(str)
test_Nae9['Predicate'] = '<' + test_Nae9['Predicate'].astype(str) + '>'
test_Nae9['Object'] = ['_:Neaf866f59ae041dd9be187c010995de0',
                       '<https://www.mirbase.org/hairpin/MI0000454>',
                       '<http://www.w3.org/2002/07/owl#NamedIndividual>']

test_Nae9

In [ ]:
subgraph = pd.concat([test_673, test_SO_0000704, test_673o, test_N7e, test_N4f, test_Nae9
          ], axis=0)
subgraph['.']='.'
subgraph.to_csv('./resources/knowledge_graphs/RNAsubgraph.nt', header=None, sep=' ', index=None)

We can copy-paste *RNAsubgraph.nt* content in https://www.ldf.fi/service/rdf-grapher to graphically visualize it.

<a target="_blank" href="https://github.com/emanuelecavalleri/testRNA-KG/assets/33032169/f769bd26-e9e1-49a8-985b-9ac3ae19f2d3"> <img src="https://github.com/emanuelecavalleri/testRNA-KG/assets/33032169/f769bd26-e9e1-49a8-985b-9ac3ae19f2d3"></a> 

(*Click Figure to Enlarge Image in Current Browser Tab*)

https://issemantic.net/rdf-visualizer allows us to dynamically inspect the graph above (in the figure below you can find a screenshot of the output).

<a target="_blank" href="https://github.com/emanuelecavalleri/testRNA-KG/assets/33032169/15fd3e41-e72d-42f8-b790-09a67916a94d"> <img src="https://github.com/emanuelecavalleri/testRNA-KG/assets/33032169/15fd3e41-e72d-42f8-b790-09a67916a94d"></a> 

(*Click Figure to Enlarge Image in Current Browser Tab*)

# OWL-NETS

From a machine learning perspective, extracting information from triples that deviate from the conventional "subject-predicate-object" pattern can be challenging due to the inclusion of biologically non-relevant information. OWL-NETS ([NEtwork Transformation for Statistical learning](https://pubmed.ncbi.nlm.nih.gov/29218876/)) is a computational method that reversibly abstracts OWL-encoded biomedical knowledge into a network representation tailored for network inference. OWL-NETS can leverage existing ontology-based knowledge representations and network inference methods to generate novel, biologically-relevant hypotheses. Further, the lossless transformation of OWL-NETS allows for seamless integration of inferred edges back into the original knowledge base, extending its coverage and completeness. 

In [ ]:
owlnets = pd.read_csv('./resources/knowledge_graphs/PheKnowLator_v3.1.1_full_instance_inverseRelations_OWLNETS.nt', sep='\s', header=None)
owlnets

In [ ]:
owlnets[owlnets[0].str.endswith("gene/673>")]

During this process, it appears that properties may be lost, but we can store them in another way.

In [ ]:
properties = pd.read_csv('./resources/knowledge_graphs/PheKnowLator_v3.1.1_full_instance_inverseRelations_OWLNETS_NodeLabels.txt', sep='\t')
properties

In [ ]:
properties[properties['entity_uri'] == '<http://www.ncbi.nlm.nih.gov/gene/673>']

In [ ]:
properties[properties['entity_uri'] == '<http://www.w3.org/2000/01/rdf-schema#subClassOf>']     

In [ ]:
properties[properties['entity_uri'] == '<http://purl.obolibrary.org/obo/RO_0002434>']

In [ ]:
list(properties[properties['entity_uri'] == '<http://purl.obolibrary.org/obo/RO_0002434>']['description/definition'])

In [ ]:
properties[properties['entity_uri'] == '<http://purl.obolibrary.org/obo/RO_0003302>']

In [ ]:
properties[properties['entity_uri'] == '<https://www.mirbase.org/hairpin/MI0000454>']

In [ ]:
properties[properties['entity_uri'] == '<http://purl.obolibrary.org/obo/MONDO_0007893>']